In [ ]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import optuna
import torch.optim as optim
import os
from tqdm import tqdm

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, json_path):
        with open(json_path, 'r') as f:
            data = json.load(f)
        self.inputs = data["inputs"]
        self.outputs = data["outputs"]

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        x = torch.tensor(self.inputs[idx], dtype=torch.float32)
        y = torch.tensor(self.outputs[idx], dtype=torch.float32)
        return x, y

def get_dataloader(json_path, batch_size=16):
    dataset = CustomDataset(json_path)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self, input_dim=3, output_dim=2):
        super(SimpleModel, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        return self.network(x)


In [ ]:
def train_model(trial):
    # ベイズ最適化でハイパーパラメータを探索
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    batch_size = trial.suggest_int('batch_size', 8, 64)
    
    model = SimpleModel()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    dataloader = get_dataloader('../data/data.json', batch_size=batch_size)

    model.train()
    best_loss = float('inf')  # 最良の損失値を保持する変数
    best_model = None  # 最良モデルを保持する変数

    num_epochs = 1000  # エポック数
    progress_bar = tqdm(range(num_epochs), desc="Training Progress", unit="epoch")  # プログレスバーの初期化
    for epoch in progress_bar:
        epoch_loss = 0
        for x, y in dataloader:
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        # 各エポック終了後に損失を計算し、最良のモデルを保存
        avg_loss = epoch_loss / len(dataloader)
        if avg_loss < best_loss:  # 最良の損失を更新
            best_loss = avg_loss
            best_model = model.state_dict()  # 最良モデルのパラメータを保存

        # プログレスバーに損失値をリアルタイム表示
        progress_bar.set_description(f"Epoch {epoch+1} | Loss: {avg_loss:.6f}")

    # 最良のモデルを保存
    if not os.path.exists('results'):
        os.makedirs('results')
    torch.save(best_model, 'results/best_model.pth')

    return best_loss  # 最良損失を返す


# Optunaのベイズ最適化
if __name__ == '__main__':
    study = optuna.create_study(direction='minimize')
    study.optimize(train_model, n_trials=1000)
    print(f"Best params: {study.best_params}")
    print(f"Best loss: {study.best_value}")


In [ ]:
def objective(trial):
    # Optunaのobjective関数として、train_model関数を使用
    return train_model(trial)

def perform_bayesian_optimization():
    # Optunaでハイパーパラメータ最適化のためのStudyを作成
    study = optuna.create_study(direction='minimize')
    
    # tqdmの進捗バーで試行回数を表示
    n_trials = 1000
    progress_bar = tqdm(total=n_trials, desc="Bayesian Optimization Progress", unit="trial")
    
    def callback(study, trial):
        # 進捗バーを更新
        progress_bar.set_description(
            f"Trial {trial.number+1} | Best Loss: {study.best_value:.6f}")
        progress_bar.update(1)

    # 最適化の実行
    study.optimize(objective, n_trials=n_trials, callbacks=[callback])
    
    # 最適な結果を表示
    progress_bar.close()
    print("Best Hyperparameters:", study.best_params)
    print("Best Loss:", study.best_value)

    # 学習済みのモデルパラメータを保存
    best_params = study.best_params
    return best_params

if __name__ == '__main__':
    best_params = perform_bayesian_optimization()
    print("Optimization completed. Best parameters:", best_params)


In [ ]:
def generate_output(input_data, model_path='results/best_model.pth'):
    model = SimpleModel()
    model.load_state_dict(torch.load(model_path, weights_only=True))
    model.eval()
    
    # 入力データをテンソルに変換し、生成
    input_tensor = torch.tensor(input_data, dtype=torch.float32)
    with torch.no_grad():
        output = model(input_tensor)
    return output.numpy()

# 生成AIテスト
if __name__ == '__main__':
    sample_input = [0.15, -0.1, 0.05]  # 例: 任意の3次元入力
    output = generate_output(sample_input)
    print(f"Generated Output: {output}")